# Data exploration

The first step of any analysis is to understand what we are searching for. In our analysis we aim to measure the central exclusive di-lepton production, $pp\to p\oplus \ell\ell \oplus p$ process with $\ell\in\{ e,\mu \} $. Feinman diagram of this process are shown bellow: 

<img src="img/diagrams.png" alt="Feinmann diagrams" style="width: 700px;"/>

Where in our case, we will consider only electrons and muons. 

---
The measurement using 2016 data was published in [JHEP07(2018)153](https://arxiv.org/abs/1803.04496). This is the first time the 2017 data will be used to measure the central exclusive di-lepton production process at higher precision.



---
<b>Remark</b>: Exclusive production of $\tau$ leptons was not measured at the LHC yet since $\tau$-leptons differ from electrons and muons by their relatively short lifetime ($c\tau_0=87\mu m$) and are observed only via their decay products. The main challenge with measuring $\tau$s is elusive neutrinos (which escape detection). Hence measurement of the momentum of $\tau$-lepton is tricky. Yet, it is possible because the opening angle between two daughter particles boosted with [lorentz factor](https://en.wikipedia.org/wiki/Lorentz_factor) $\gamma$ is given by $\theta ~\sim 2/\gamma$. With a large enough boost, the opening angle became collinear, and neutrino 4-momentum can be measured.

To understand the process better, we will explore the final state signature. As we discussed earlier, samples are stored in the `h5py` data format, which can be easily accessed with Jupyter notebook. 


In [ ]:
#start with standard python imports
#!pip install --user mplhep #enable this line if the import of mplhep fails
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib.colors import LogNorm

In [ ]:
#to make the plots in CMS style execute this line
plt.style.use([hep.style.ROOT, hep.style.firamath])
plt.style.use(hep.style.CMS)

In [ ]:
#Execute this line if running on SWAN, otherwise update the path to the data files:
PATH='/eos/user/c/cmsdas/short-exercises/pps-protons-tutorial/data'
#PATH='output'

## Loading the data (signal)

We will load `h5py` files of the simulated signal events. Note that three different central exclusive di-lepton production processes are considered: exclusive, semi-exclusive, and inclusive (see Figure 1). We will load the files and convert them to pandas datafrme. Let's explore the differences between the processes.

### Dataformat:

We will use the following code `GetData(filename.h5)` to read the `h5` file and convert the data to pandas dataframe.

In [ ]:
def GetData(filename):
    
    """ opens a summary file and converts it to a pandas dataframe """
    
    with h5py.File(filename, 'r') as f:
        dset = f['protons']
        dset_columns = f['columns']
        columns = list( dset_columns )
        columns_str = [ item.decode("utf-8") for item in columns ]
        return pd.DataFrame( dset, columns=columns_str )
    
    return pd.DataFrame()

In [ ]:
#load the signal samples into the dataframes (takes some time)
df_signal_excl = GetData(PATH+'/output-MC2017-Elastic-Non3+3-PreSel.h5')
df_signal_semiexcl = GetData(PATH+'/output-MC2017-SingleDissociation-Non3+3-PreSel.h5')

In [ ]:
#Load the data samples into the dataframes (takes some time)
df_data={}
eras=['B']
#eras=['B','C1','E','F1'] #uncooment to process all data
for x in eras:
    df_data[x] = GetData(PATH+'/output-UL2017{}-PreSel.h5'.format(x))
    df_data[x]['era']=x
    print('output-UL2017{}-PreSel shape = {}'.format(x,df_data[x].shape))

#combine all into a single one
df_data=pd.concat([df_data[x] for x in eras])

### Exploring the data files

Similarly to what we did with ROOT files in the short exercise, let's look at the info we have in the dataframes:

In [ ]:
def PrintInfoFromDF(df):
    print('Print all branches:')
    print(df.keys())
    print('Size of the data is ',df.shape)

In [ ]:
PrintInfoFromDF(df_signal_excl)
PrintInfoFromDF(df_data)

As you can see, we have 38 different columns and 212744 raws in the file (each raw corresponds to a different event). In data we have added an extra column to flag which data-taking era is the event comming from.

**TASK 1**

Look at distributions of different kinematic variables (among different processes) and try to see if you observe any difference... The code below `PlotFromDF(variable, dataframes, labels)` will plot normalized shapes of selected variables. 


In [ ]:
def PlotFromDF(variable, xmin, xmax, nbins, dataframes, _labels, ax, log=False):
    bins = np.linspace(xmin,xmax,nbins)
    data=[]; labels=[]
    for df, label in zip(dataframes, _labels):
        h, _ = np.histogram(df[variable], bins,density=True)
        data.append(h)
        labels.append(label)
    hep.histplot(data, bins, ax=ax, label=labels)
    hep.cms.label(data=True, paper=False, year='2017', ax=ax)
    ax.legend(); 
    ax.set(xlabel=variable, ylabel='p.d.f.')
    if log: plt.yscale("log")

In the following example, we will plot the di-lepton [acoplanarity](https://en.wikipedia.org/wiki/Acoplanarity) defined by:
$$A = 1 - \Delta\phi(\mu,\mu)/\pi$$

In the exclusive events, due to absence of additional radiation, both leptons expected to be produced back-to-back, or with $\Delta\phi(\mu,\mu)\sim\pi$. 

In [ ]:
# we will be plotting MC prediction with the data, where the data is mostly populated with background events
procc = [df_signal_excl,df_signal_semiexcl,df_data]
labels = ['Exclusive dilep','Semi-exclusive dilep','data']

In [ ]:
f, ax = plt.subplots()
PlotFromDF('Acopl',0,1,100,procc,labels, ax, log=True)

In [ ]:
#f, ax = plt.subplots()
#PlotFromDF('ExtraPfCands',0,20,20,procc,labels, ax, log=False)

In [ ]:
#f, ax = plt.subplots()
#PlotFromDF('InvMass',0,800,100,procc,labels, ax, log=True)

### Tagged protons

In central exclusive events, the tagged protons' momentum loss corresponds to the 4-momentum of the interacting photons. From proton momentum loss, denoted by $\xi=1-\Delta p/p$, we can reconstruct two quantities of the central system - the mass and the rapidity:

\begin{equation}
\begin{split}
m_{\ell\ell} &= \sqrt{s\xi_1\xi_2}\\
Y_{\ell\ell} &= \frac{1}{2}\log(\xi_1/\xi_2),
\end{split}\label{eq:eq1}
\end{equation}
In case of single dissociation (when only one proton remains intact), we can reconstruct proton momentum loss from the kinematics measured in the central detector: di-lepton mass $m_{\ell\ell}$ and pseudo-rapidity of the leptons $\eta_{\ell\pm}$, using the following formula:

\begin{equation}
\xi_\pm = \frac{1}{\sqrt{s}}[p_T(\ell+)e^{\pm\eta(\ell+)} + p_T(\ell-)e^{\pm\eta(\ell-)}] 
\label{eq:eq2}
\end{equation}


In the next block we will inspect this correlation

<b>TASK 2</b>

Write a function that will produce a 2D scatter plot of measured $\xi$ in the forward detector and the reconstructed $\xi(\ell\ell)$ using the second formula above.


HINT: Use the block below to add a new column to the dataframe with reconstructed $\xi$ named `recXi_pos` and `recXi_neg`
```python
df_signal['recXi_pos'] = USE FORMULA (2) TO COMPUTE NEGATIVE AND POSSITIVE XIs
df_signal['recXi_neg'] = USE FORMULA (2) TO COMPUTE NEGATIVE AND POSSITIVE XIs
```


In [ ]:
def ComputeXi(dataframe):
    
    """ computes the proton csi based on the central system kinematics"""
    
    dataframe['recXi_pos'] = dataframe['XiMuMuPlus'] # USE FORMULA (2) TO COMPUTE NEGATIVE AND POSSITIVE XIs
    dataframe['recXi_neg'] = dataframe['XiMuMuMinus'] # USE FORMULA (2) TO COMPUTE NEGATIVE AND POSSITIVE XIs
    
    return dataframe


#if you are stuck uncomment the following line to load the solution
#%load ./snippets/DataInspectionTask2.py

The code below `Plot2DScatter(dataframe)` will plot 2D scatter plot between reconstructed $\xi$ from the two leptons and $\xi$ measured by the forward detectors:

In [ ]:
def Plot2DScatter(dataframe, mask = [], proton_selection = "MultiRP"):

    """Shows the corelation between the xi reconstucted from the PPS system and from the central dilepton system"""
    
    if not mask.empty and len(mask) != dataframe.shape[0]:
        print('Error: bad mask, check if the mask corresponds to the dataframe')
        return
    
    if proton_selection == "SingleRP":
        # Single-RP in pixel stations
        msk1 = mask & ( dataframe["MultiRP"] == 0) & ( dataframe["RPId1"] == 23 )
        msk2 = mask & ( dataframe["MultiRP"] == 0) & ( dataframe["RPId1"] == 123 )
    elif proton_selection == "MultiRP":
        # Multi-RP
        msk1 = mask & ( dataframe["MultiRP"] == 1 ) & ( dataframe["Arm"] == 0 )
        msk2 = mask & ( dataframe["MultiRP"] == 1 ) & ( dataframe["Arm"] == 1 )
 
    print ( len(dataframe[ "Xi" ][ msk1 ]), len(dataframe[ "Xi" ][ msk2 ]) )

    fig, axes = plt.subplots( 2, 2, figsize=(20,20) )
    axes[0,0].plot( dataframe[ "Xi" ][ msk1 ], dataframe[ "recXi_pos" ][ msk1 ], 'ko' )
    axes[0,1].plot( dataframe[ "Xi" ][ msk2 ], dataframe[ "recXi_neg" ][ msk2 ], 'ko' )
    _,_,_, im = axes[1,0].hist2d( dataframe[ "Xi" ][ msk1 ], dataframe[ "recXi_pos" ][ msk1 ], bins=(50,50), norm=LogNorm(), cmap='viridis' )
    fig.colorbar(im, ax=axes[1,0])
    _,_,_, im = axes[1,1].hist2d( dataframe[ "Xi" ][ msk2 ], dataframe[ "recXi_neg" ][ msk2 ], bins=(50,50), norm=LogNorm(), cmap='viridis' )
    fig.colorbar(im, ax=axes[1,1])

    for i in range(2):
        for j in range(2):
            #aux lines
            axes[i,j].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
            axes[i,j].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
            axes[i,j].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
            axes[i,j].set_xlim(0.,0.15)
            axes[i,j].set_ylim(0.,0.15)
            sgn='+' if j==0 else '-'
            axes[i,j].set_xlabel(r'$\xi_{%s}$ (PPS)'%(sgn))
            axes[i,j].set_ylabel(r'$\xi_{%s}$ (dileptons)'%(sgn))

### Correlation plots - signal:

Check your results with ploting correlation plots for the signal (exclusive) events:

In [ ]:
#before the start, lets mask events with high invariant mass, with two leptons produced back to back exclusivelly 
df_signal = df_signal_excl
msk_excl = ( df_signal["InvMass"] >= 110. ) & ( df_signal["Acopl"] <= 0.009 ) & ( df_signal["ExtraPfCands"] <= 1 )

#in addiitonl we will focus only on MultiRP reconstruction
proton_selection = "MultiRP" # "SingleRP" or "MultiRP"

#compute xi
ComputeXi(df_signal)

#plot correlations:
Plot2DScatter(df_signal, mask=msk_excl, proton_selection=proton_selection)

We can also plot the difference betweeen recontructed and measured $\xi$ values in 1D plot. It is easier to mask only

In [ ]:
# add new variables to dataframes for possitivle and negative xi's :
mask_pos = msk_excl & ( df_signal["MultiRP"] == 1 ) & ( df_signal["Arm"] == 0 )
mask_neg = msk_excl & ( df_signal["MultiRP"] == 1 ) & ( df_signal["Arm"] == 1 )

df_signal['delxi_pos'] = (1 - df_signal[ "Xi" ] / df_signal[ "recXi_pos" ])
df_signal['delxi_neg'] = (1 - df_signal[ "Xi" ] / df_signal[ "recXi_neg" ])

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )
PlotFromDF('delxi_pos',-5,3,100,[df_signal[ mask_pos ]],['exclusive di-$\ell$'],axes[0], log=False)
PlotFromDF('delxi_neg',-5,3,100,[df_signal[ mask_neg ]],['exclusive di-$\ell$'],axes[1], log=False)

### Correlation plots - data:

<b>TASK 3</b>

Produce correlation plots now for data events as we did for the signal

In [ ]:
#MASK EVENTS WITH TWO LEPTONS PRODUCED BACK TO BACK EXCLUSEVILY
msk_data = ( df_data["InvMass"] >= 110. ) & ( df_data["Acopl"] <= 0.009 ) & ( df_data["ExtraPfCands"] <= 1 )

#in addiitonl we will focus only on MultiRP reconstruction
proton_selection = "MultiRP" # "SingleRP" or "MultiRP"

#compute xi
ComputeXi(df_data)

#plot correlations:
Plot2DScatter(df_data, mask=msk_data, proton_selection=proton_selection)

### Compare data to signal + background

In the following we'll use the data as a background (assuming we haven't yet selected any signal candidate).
We'll make a plot of the difference between the $\xi$ reconstructed from PPS and the one reconstructed from the dilepton system.

In [ ]:
df_bkg = df_data.copy()

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

bins_ = 40
range_ = (-5.,3.)
resample_factor = 680


counts={}
errors={}
for arm in [0,1]:

    xi_dil='XiMuMuPlus' if arm==0 else 'XiMuMuMinus'
    
    #build the differences for the data
    msk_data_arm = msk_data & ( df_data["MultiRP"] == 1 ) & ( df_data["Arm"] == arm )
    delta = ( 1. - df_data[ "Xi" ][ msk_data_arm ] / df_data[ xi_dil ][ msk_data_arm ] )
    counts[arm], bin_edges = np.histogram( delta, bins=bins_, range=range_ )
    errors[arm] = np.sqrt( counts[arm] )
    bin_centers = ( bin_edges[:-1] + bin_edges[1:] ) / 2.
    axes[arm].errorbar(bin_centers, counts[arm], yerr=errors[arm], fmt='o', label='Data')
        
    print('Arm=',arm,'Data counts=',counts[arm], 'bin edges=',bin_edges )

    #build the differences for the background
    msk_bkg = ( df_bkg["MultiRP"] == 1 ) & ( df_bkg["Arm"] == arm )
    weights = None
    if resample_factor > 1:
        weights = np.full_like( df_bkg[ "Xi" ][ msk_bkg ], ( 1./resample_factor ) )
    delta_bkg = ( 1. - df_bkg[ "Xi" ][ msk_bkg ] / df_bkg[ xi_dil ][ msk_bkg ] )
    axes[arm].hist( delta_bkg, bins=bins_, range=range_, weights=weights, label='Background' )

    
#final tweak to have both y-axis at the same scale showing all data
total_counts=np.concatenate( [counts[0],counts[1]] )
total_errors=np.concatenate( [errors[0], errors[1]] )
idx_ymax_ = np.argmax( total_counts )
y_max = total_counts[idx_ymax_] + 2*total_errors[idx_ymax_]
print ( "y max. = {}".format(y_max) )
for arm in [0,1]:
    axes[arm].set_ylim( top=y_max )
    sgn='+' if arm==0 else '-'
    axes[arm].set_xlabel(r'$[\Delta\xi/\xi(\ell\ell)]_{%s}$'%(sgn))
    axes[arm].set_ylabel('Events')
    axes[arm].legend(loc='best')

## Ancillary variables

In the following we plot ancillary variables, e.g. $\theta_y$, applying a cut on the resolution of the reconstructed $\xi$.

In [ ]:
def plotAncillary(data, bkg, var='ThY', rangeDelta=(-0.20,0.20), bins=10, xran=(-0.005,0.005) ):
    
    """ compares data and background for the variable var applying a cut on the relative resolution defined by rangeDelta """

    fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

    
    counts={}
    errors={}
    for arm in [0,1]:
    
        xi_dil='XiMuMuPlus' if arm==0 else 'XiMuMuMinus'
    
        #mask the data to be plotted
        msk_data_arm = msk_data & ( data["MultiRP"] == 1 ) & ( data["Arm"] == arm )
        delta = ( 1. - data[ "Xi" ][ msk_data_arm ] / data[ xi_dil ][ msk_data_arm ] )
        msk_data_delta = (delta>=rangeDelta[0]) & (delta<=rangeDelta[1])
        counts[arm], bin_edges = np.histogram( data[var][msk_data_arm][msk_data_delta], bins=bins, range=xran )
        errors[arm] = np.sqrt( counts[arm] )
        bin_centres = ( bin_edges[:-1] + bin_edges[1:] ) / 2.
        axes[arm].errorbar(bin_centres, counts[arm], yerr=errors[arm], fmt='o',label='Data')
        
        print('Arm=',arm,'Data counts=',counts, 'bin edges=',bin_edges )

        #mask the background
        msk_bkg = ( bkg["MultiRP"] == 1 ) & ( bkg["Arm"] == arm )
        weights = None
        if resample_factor > 1:
            weights = np.full_like( bkg[ var ][ msk_bkg ], ( 1./resample_factor ) )
       
        vals_bkg = bkg[msk_bkg]
        delta_bkg = ( 1. - vals_bkg[ "Xi" ] / vals_bkg[xi_dil ] )
        msk_bkg_delta = (delta_bkg>=rangeDelta[0]) & (delta_bkg<=rangeDelta[1])
            
        axes[arm].hist( vals_bkg[var][msk_bkg_delta], bins=bins, range=xran, weights=weights[msk_bkg_delta], label='Background' )

    #final tweak to have both y-axis at the same scale showing all data
    total_counts=np.concatenate( [counts[0],counts[1]] )
    total_errors=np.concatenate( [errors[0], errors[1]] )
    idx_ymax_ = np.argmax( total_counts )
    y_max = total_counts[idx_ymax_] + 2*total_errors[idx_ymax_]
    print ( "y max. = {}".format(y_max) )
    for arm in [0,1]:
        axes[arm].set_ylim( top=y_max )
        sgn='+' if arm==0 else '-'
        axes[arm].set_xlabel(var)
        axes[arm].set_ylabel('Events')
        axes[arm].legend(loc='best')
    

deltaXi_cut=(-0.20,0.20)
#plotAncillary(df_data,df_bkg, var='Xi',  rangeDelta=deltaXi_cut ,bins=10,xran=(0.,0.2))
#plotAncillary(df_data,df_bkg, var='ThX', rangeDelta=deltaXi_cut ,bins=10,xran=(-0.0005,0.0005))
plotAncillary(df_data,df_bkg, var='ThY', rangeDelta=deltaXi_cut ,bins=10,xran=(-0.005,0.005))
#plotAncillary(df_data,df_bkg, var='T',   rangeDelta=deltaXi_cut ,bins=10,xran=(-4,0))
#plotAncillary(df_data,df_bkg, var='Time',rangeDelta=deltaXi_cut ,bins=10,xran=(-0.5,0.5))